# HeudiConv Basics: DICOM to BIDS Conversion

This notebook demonstrates how to use the HeudiConv runner to convert DICOM files to BIDS format.

## Overview

HeudiConv is a flexible DICOM converter that creates BIDS-compliant datasets. It uses heuristic files to map DICOM series to BIDS-compliant names.

## Prerequisites

- Docker installed and running
- DICOM data organized by participant
- A heuristic file defining the conversion rules

## Setup

In [1]:
from pathlib import Path
import pandas as pd
from voxelops import (
    HeudiconvDefaults,
    HeudiconvInputs,
    run_heudiconv,
    run_procedure
)

In [10]:
from voxelops.procedures.orchestrator import _get_default_log_dir
import json

def _get_last_execution_log(inputs: HeudiconvInputs, log_dir: Path | None = None) -> bool:
    """Return the path to the last execution log for a participant/session.

    Parameters
    ----------
    inputs : HeudiconvInputs
        The inputs for the heudiconv procedure.
    log_dir : Path
        Directory containing execution logs.
    log_dir : Path
        Directory containing execution logs.

    Returns
    -------
    Path | None
        Path to the last execution log, or None if not found.
    """
    if (log_dir is None) or (not log_dir.exists()):
        log_dir = _get_default_log_dir(inputs)
        

    prefix = f"heudiconv_sub-{inputs.participant}"
    if inputs.session is not None:
        prefix += f"_ses-{inputs.session}"

    log_files = list(log_dir.glob(f"{prefix}_*.json"))
    if not log_files:
        return False
    last_executed = sorted(log_files, key=lambda f: f.stat().st_mtime)[-1]

    # get "success" field from the log
    with open(last_executed) as f:
        log_data = json.load(f)
        print(log_data.get("success"))
        if log_data.get("success"):
            return True
        else:
            return False

In [11]:

from typing import Union
import json

def sanitize_subject_code(subject_code: str) -> str:
    """Remove special characters and zero-pad to 4 digits."""
    return subject_code.replace("-", "").replace("_", "").replace(" ", "").zfill(4)


def sanitize_session_id(session_id: Union[str, int, float]) -> str:
    """Convert to string, clean, and zero-pad to 12 digits."""
    if isinstance(session_id, float):
        if pd.isna(session_id):
            return ""
        session_str = str(int(session_id))
    else:
        session_str = str(session_id)
    return session_str.replace("-", "").replace("_", "").replace(" ", "").zfill(12)


def load_sessions(csv_path: Union[str, Path]) -> pd.DataFrame:
    """Load and sanitize a linked_sessions CSV file.

    Expects columns SubjectCode and ScanID. Returns a DataFrame with
    ``subject_code`` and ``session_id`` columns (sanitized, de-duplicated).

    Parameters
    ----------
    csv_path:
        Path to the linked_sessions.csv file.
    """
    df = pd.read_csv(csv_path)
    df["subject_code"] = df["SubjectCode"].apply(sanitize_subject_code)
    df["session_id"] = df["ScanID"].apply(sanitize_session_id)
    return df.drop_duplicates(subset=["subject_code", "session_id"]).reset_index(drop=True)

sessions = load_sessions("~/Downloads/linked_sessions.csv")
sessions = sessions[sessions["dicom_path"].notna()]

config = HeudiconvDefaults(
    heuristic=Path("/home/galkepler/Projects/yalab-devops/VoxelOps/heuristic.py"),
    overwrite=False
)

log_dir = None

for idx, row in sessions.iterrows():
    print(f"Processing subject {row['subject_code']} session {row['session_id']} with DICOM path {row['dicom_path']}")
    inputs = HeudiconvInputs(
        dicom_dir=Path(row["dicom_path"]),
        participant=row["subject_code"],
        session=row["session_id"],
        output_dir=Path("/media/storage/yalab-dev/qsiprep_test/BIDS"),
        
    )
    executed_successfully = _get_last_execution_log(inputs, log_dir)
    # result = run_procedure(
    #     procedure="heudiconv",
    #     inputs=inputs,
    #     config=config,
    #     )
    # result = run_heudiconv(inputs, overwrite=False, heuristic=Path("/home/galkepler/Projects/yalab-devops/VoxelOps/heuristic.py"))
    break

Processing subject YA1397 session 202602161650 with DICOM path /mnt/62/Raw_Data/20260216_1650
True


In [7]:
executed_successfully

False

In [22]:
result

{'tool': 'heudiconv',
 'participant': 'YA1409',
 'command': ['docker',
  'run',
  '--rm',
  '--user',
  '1000:1000',
  '-v',
  '/mnt/62/Raw_Data/20260211_1818:/dicom:ro',
  '-v',
  '/media/storage/yalab-dev/qsiprep_test/BIDS:/output',
  '-v',
  '/home/galkepler/Projects/yalab-devops/VoxelOps/heuristic.py:/heuristic.py:ro',
  'nipy/heudiconv:1.3.4',
  '--files',
  '/dicom',
  '--outdir',
  '/output',
  '--subjects',
  'YA1409',
  '--converter',
  'dcm2niix',
  '--heuristic',
  '/heuristic.py',
  '--ses',
  '202602111818',
  '--overwrite',
  '--bids',
  '--bids',
  'notop',
  '--grouping',
  'all'],
 'exit_code': 0,
 'start_time': '2026-02-19T11:48:35.772048',
 'end_time': '2026-02-19T11:59:08.931295',
 'duration_seconds': 633.159247,
 'duration_human': '0:10:33.159247',
 'success': True,
 'stdout': '(\'ORIGINAL\', \'PRIMARY\', \'M\', \'DIS2D\')\n(\'ORIGINAL\', \'PRIMARY\', \'M\', \'NORM\', \'DIS2D\')\n260219-09:51:21,478 nipype.workflow INFO:\n\t [Node] Setting-up "convert" in "/tmp/dcm

In [16]:
result

{'tool': 'heudiconv',
 'participant': 'YA1409',
 'command': ['docker',
  'run',
  '--rm',
  '--user',
  '1000:1000',
  '-v',
  '/mnt/62/Raw_Data/20260211_1818:/dicom:ro',
  '-v',
  '/media/storage/yalab-dev/qsiprep_test/BIDS:/output',
  '-v',
  '/home/galkepler/Projects/yalab-devops/VoxelOps/heuristic.py:/heuristic.py:ro',
  'nipy/heudiconv:1.3.4',
  '--files',
  '/dicom',
  '--outdir',
  '/output',
  '--subjects',
  'YA1409',
  '--converter',
  'dcm2niix',
  '--heuristic',
  '/heuristic.py',
  '--ses',
  '202602111818',
  '--overwrite',
  '--bids',
  '--bids',
  'notop',
  '--grouping',
  'all'],
 'exit_code': 0,
 'start_time': '2026-02-19T11:48:35.772048',
 'end_time': '2026-02-19T11:59:08.931295',
 'duration_seconds': 633.159247,
 'duration_human': '0:10:33.159247',
 'success': True,
 'stdout': '(\'ORIGINAL\', \'PRIMARY\', \'M\', \'DIS2D\')\n(\'ORIGINAL\', \'PRIMARY\', \'M\', \'NORM\', \'DIS2D\')\n260219-09:51:21,478 nipype.workflow INFO:\n\t [Node] Setting-up "convert" in "/tmp/dcm

## Define Paths

Update these paths to match your data structure:

In [ ]:
# Input paths -- update these to match your data
dicom_dir = Path("/data/raw/dicom/")
participant = "01"
heuristic_file = Path("/config/heuristics/brain_bank.py")

# Output paths (optional -- will use defaults if not specified)
output_dir = Path("/data/bids/")

## Basic Usage

### Option 1: Use Default Configuration

In [3]:
# Create inputs
inputs = HeudiconvInputs(
    dicom_dir=dicom_dir,
    participant=participant,
    session="01",
    output_dir=output_dir,
    # work_dir=work_dir,
)

In [ ]:
# Run with defaults
result = run_heudiconv(inputs, heuristic=heuristic_file, overwrite=True)

# Check result
print(f"Success: {result['success']}")
print(f"Duration: {result['duration_human']}")
print(f"Output: {result['expected_outputs'].bids_dir}")

In [ ]:
# Run with parameter overrides
result = run_heudiconv(
    inputs,
    overwrite=True,  # Overwrite existing output
    bids_validate=True,  # Run BIDS validator
    docker_image="nipy/heudiconv:0.13.1",  # Use specific version
)

print(f"Success: {result['success']}")
print(f"Exit code: {result['exit_code']}")

### Option 3: Use Custom Configuration

In [ ]:
# Run with parameter overrides
result = run_heudiconv(
    inputs,
    overwrite=True,
    bids_validator=True,  # Run BIDS validator after conversion
    docker_image="nipy/heudiconv:1.3.4",  # Pin specific version
)

print(f"Success: {result['success']}")
print(f"Exit code: {result['exit_code']}")

## Inspect Execution Record

The execution record contains complete information about the run:

In [ ]:
# Create custom configuration
config = HeudiconvDefaults(
    overwrite=True,
    bids_validator=True,
    docker_image="nipy/heudiconv:1.3.4",
)

# Run with custom config
result = run_heudiconv(inputs, config)

print(f"Success: {result['success']}")

## Check Expected Outputs

In [ ]:
outputs = result["expected_outputs"]

print("Expected outputs:")
print(f"BIDS directory: {outputs.bids_dir}")
print(f"Participant directory: {outputs.participant_dir}")
print(f"Work directory: {outputs.work_dir}")

# Check if outputs exist
print("\nOutput validation:")
print(f"BIDS dir exists: {outputs.bids_dir.exists()}")
print(f"Participant dir exists: {outputs.participant_dir.exists()}")

# List files in participant directory
if outputs.participant_dir.exists():
    print(f"\nFiles in {outputs.participant_dir}:")
    for f in outputs.participant_dir.rglob("*"):
        if f.is_file():
            print(f"  {f.relative_to(outputs.participant_dir)}")

## Save Execution Record

Save the execution record to a database or file:

In [ ]:
outputs = result["expected_outputs"]

print("Expected outputs:")
print(f"  BIDS directory: {outputs.bids_dir}")
print(f"  Participant directory: {outputs.participant_dir}")
print(f"  Dataset description: {outputs.dataset_description}")

# Check if outputs exist
print("\nOutput validation:")
print(f"  BIDS dir exists: {outputs.bids_dir.exists()}")
print(f"  Participant dir exists: {outputs.participant_dir.exists()}")

# List files in participant directory
if outputs.participant_dir.exists():
    print(f"\nFiles in {outputs.participant_dir}:")
    for f in outputs.participant_dir.rglob("*"):
        if f.is_file():
            print(f"  {f.relative_to(outputs.participant_dir)}")

## Error Handling

In [ ]:
from voxelops.exceptions import (
    InputValidationError,
    ProcedureExecutionError,
)

try:
    result = run_heudiconv(inputs)
    print(f"Success: {result['success']}")
except InputValidationError as e:
    print(f"Input validation failed: {e}")
except ProcedureExecutionError as e:
    print(f"Execution failed: {e}")
    print(f"Check logs at: {result.get('log_file')}")
except Exception as e:
    print(f"Unexpected error: {e}")

## Batch Processing Multiple Participants

In [ ]:
# List of participants to process
participants = ["01", "02", "03", "04", "05"]

results = []

for participant in participants:
    print(f"\nProcessing participant {participant}...")

    inputs = HeudiconvInputs(
        dicom_dir=dicom_dir,
        participant=participant,
        heuristic=heuristic_file,
        output_dir=output_dir,
    )

    try:
        result = run_heudiconv(inputs)
        results.append(result)
        print(f"  ✓ Success in {result['duration_human']}")
    except Exception as e:
        print(f"  ✗ Failed: {e}")
        results.append({"participant": participant, "success": False, "error": str(e)})

# Summary
successful = sum(1 for r in results if r.get("success"))
print(
    f"\nProcessed {len(results)} participants: {successful} successful, {len(results) - successful} failed"
)

## Next Steps

After converting DICOM to BIDS:

1. Validate the BIDS dataset using the BIDS validator
2. Run quality control checks
3. Proceed to preprocessing with QSIPrep (see `02_qsiprep_basics.ipynb`)

## Tips

- **Heuristic files**: Create a reusable heuristic file for your study protocol
- **BIDS validation**: Enable `bids_validate=True` to catch errors early
- **Overwrite carefully**: Use `overwrite=True` only when necessary
- **Check logs**: If conversion fails, check the log file for detailed error messages
- **Docker image**: Pin a specific version for reproducibility